## Loading the splits

In [18]:
import pandas as pd
import numpy as np
import warnings
#import mlflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [19]:
# loading the data splits

from pathlib import Path

split_dir = Path("../data/splits")

train_paths  = np.load(split_dir / "train_paths.npy", allow_pickle=True)
train_labels = np.load(split_dir / "train_labels.npy", allow_pickle=True)

val_paths  = np.load(split_dir / "val_paths.npy", allow_pickle=True)
val_labels = np.load(split_dir / "val_labels.npy", allow_pickle=True)

test_paths  = np.load(split_dir / "test_paths.npy", allow_pickle=True)
test_labels = np.load(split_dir / "test_labels.npy", allow_pickle=True)

print(len(train_paths), len(val_paths), len(test_paths))
print(train_paths[0], train_labels[0])


14034 3509 3096
..\data\raw\PlantVillage\YellowLeaf__Curl_Virus\60d14bc3-b703-4b83-8bf9-f13124970145___YLCV_GCREC 2934.JPG 7


In [20]:
#Loading the class names

import json

with open(split_dir / "class_names.json", "r") as f:
    class_names = json.load(f)

for i, name in enumerate(class_names):
    print(f"{i}: {name}")


0: Bacterial_spot
1: Early_blight
2: Late_blight
3: Leaf_Mold
4: Septoria_leaf_spot
5: Spider_mites_Two_spotted_spider_mite
6: Target_Spot
7: YellowLeaf__Curl_Virus
8: healthy
9: mosaic_virus


In [ ]:
# Build TensorFlow datasets from the indices

import tensorflow as tf

IMG_SIZE = (224, 224) #for EfficientNetB0
BATCH_SIZE = 32
SEED = 12

def load_image(path, label):
    img = tf.io.read_file(path) #reads the image file
    img = tf.image.decode_image(img, channels=3, expand_animations=False) #decodes images into uint8 tensor, RGB channels
    img = tf.image.resize(img, IMG_SIZE) #resizes images to specified size
    img = tf.cast(img, tf.float32)  # keep [0..255]
    img.set_shape([IMG_SIZE[0], IMG_SIZE[1], 3]) # forces static shape of the tensor
    return img, label #returns image and label in a format suitable for Keras

def make_dataset(paths, labels, shuffle=False):
    paths = np.array([str(p) for p in paths])  # convert WindowsPath -> str
    ds = tf.data.Dataset.from_tensor_slices((paths, labels)) #creates a dataset where each element is (path, label)
    ds = ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE) #each element is processed by load_image function
    ds = ds.ignore_errors() #ignores errors during data loading
    if shuffle: # shuffle is for training dataset only, prevents from seeing the data in the same order every epoch
        ds = ds.shuffle(2000, seed=SEED, reshuffle_each_iteration=True)
    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE) #groups samples into batches and fetches them in the background
    return ds


pv_train_ds = make_dataset(train_paths, train_labels, shuffle=True)
pv_val_ds   = make_dataset(val_paths, val_labels)
pv_test_ds  = make_dataset(test_paths, test_labels)

x, y = next(iter(pv_train_ds))
print(x.shape, y.shape, x.dtype)

(32, 224, 224, 3) (32,) <dtype: 'float32'>


In [22]:
print(train_paths[:3])

[WindowsPath('../data/raw/PlantVillage/YellowLeaf__Curl_Virus/60d14bc3-b703-4b83-8bf9-f13124970145___YLCV_GCREC 2934.JPG')
 WindowsPath('../data/raw/PlantVillage/Late_blight/3c20c90a-788c-4d06-acdb-107f695d901b___RS_LB 4856.JPG')
 WindowsPath('../data/raw/PlantVillage/YellowLeaf__Curl_Virus/172cb996-a7ad-45a3-9d3b-0be425414d94___YLCV_GCREC 2323.JPG')]


Now we add the wild train/val data. Wild test data is already stored in ..data/wild_test.

In [23]:
import numpy as np
from pathlib import Path

split_dir = Path("../data/splits")

wild_train_paths  = np.load(split_dir / "wild_train_paths.npy", allow_pickle=True)
wild_train_labels = np.load(split_dir / "wild_train_labels.npy")

wild_val_paths  = np.load(split_dir / "wild_val_paths.npy", allow_pickle=True)
wild_val_labels = np.load(split_dir / "wild_val_labels.npy")

wild_train_ds = make_dataset(wild_train_paths, wild_train_labels, shuffle=True)
wild_val_ds   = make_dataset(wild_val_paths, wild_val_labels)


We create a mixed train dataset from the original and wild dataset. Same goes for validation dataset. In both cases, the wild data is strongly weighted.

In [33]:
mixed_train_ds = tf.data.Dataset.sample_from_datasets(
    [pv_train_ds, wild_train_ds],
    weights=[0.3, 0.7],
    seed=SEED
)

val_ds = tf.data.Dataset.sample_from_datasets(
    [pv_val_ds, wild_val_ds],
    weights=[0.3, 0.7],
    seed=SEED
)


In [34]:
x, y = next(iter(mixed_train_ds))
print(x.shape, y.shape, x.dtype, y.dtype)


(32, 224, 224, 3) (32,) <dtype: 'float32'> <dtype: 'int32'>


## Building the model

1) Augmentation

In [26]:
import tensorflow as tf
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.06),
    tf.keras.layers.RandomZoom(0.15),
    tf.keras.layers.RandomTranslation(0.08, 0.08),
    tf.keras.layers.RandomContrast(0.25),
    tf.keras.layers.RandomBrightness(0.20),
    tf.keras.layers.GaussianNoise(0.03),
], name="augmentation")



In [ ]:
# to deal with hand occlusions, etc.
# layers.RandomErasing = getattr(layers, "RandomErasing", None)
#if layers.RandomErasing is not None:
#    data_augmentation.add(layers.RandomErasing(factor=0.15))


2) EfficientNet preprocessing

In [27]:
#preprocessing layer for EfficientNet
from tensorflow.keras.applications.efficientnet import preprocess_input
preprocess = layers.Lambda(preprocess_input, name="preprocess_input")


3) Build the model (EfficientNetB0 default)

In [28]:
from tensorflow.keras import models

NUM_CLASSES = len(class_names)
IMG_SIZE = (224, 224)

base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(*IMG_SIZE, 3),
    pooling="avg"
)
base_model.trainable = False  # phase 1: freeze backbone


inputs = layers.Input(shape=(*IMG_SIZE, 3))
x = data_augmentation(inputs)
x = preprocess(x)
x = base_model(x, training=False)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = models.Model(inputs, outputs)
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 augmentation (Sequential)   (None, 224, 224, 3)       0         
                                                                 
 preprocess_input (Lambda)   (None, 224, 224, 3)       0         
                                                                 
 efficientnetb0 (Functional  (None, 1280)              4049571   
 )                                                               
                                                                 
 dropout_1 (Dropout)         (None, 1280)              0         
                                                                 
 dense_1 (Dense)             (None, 10)                12810     
                                                           

4) Compile + callbacks

In [29]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)


In [31]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

callbacks = [
    ModelCheckpoint("../models/v3_best.keras", monitor="val_loss", mode="min", save_best_only=True),
    EarlyStopping(monitor="val_loss", mode="min", patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", mode="min", factor=0.5, patience=2, min_lr=1e-6, verbose=1),
]


5) Train phase 1 (with frozen backbone)

In [35]:
history1 = model.fit(
    mixed_train_ds,
    validation_data=val_ds,
    epochs=8,
    callbacks=callbacks
)


Epoch 1/8
444/444 [==============================] - 276s 606ms/step - loss: 1.0273 - accuracy: 0.6923 - val_loss: 0.6315 - val_accuracy: 0.8188 - lr: 0.0010
Epoch 2/8
444/444 [==============================] - 264s 594ms/step - loss: 0.6177 - accuracy: 0.8158 - val_loss: 0.4853 - val_accuracy: 0.8524 - lr: 0.0010
Epoch 3/8
444/444 [==============================] - 261s 587ms/step - loss: 0.5165 - accuracy: 0.8457 - val_loss: 0.4435 - val_accuracy: 0.8575 - lr: 0.0010
Epoch 4/8
444/444 [==============================] - 259s 583ms/step - loss: 0.4676 - accuracy: 0.8572 - val_loss: 0.3996 - val_accuracy: 0.8710 - lr: 0.0010
Epoch 5/8
444/444 [==============================] - 263s 590ms/step - loss: 0.4387 - accuracy: 0.8648 - val_loss: 0.3730 - val_accuracy: 0.8789 - lr: 0.0010
Epoch 6/8
444/444 [==============================] - 260s 584ms/step - loss: 0.4054 - accuracy: 0.8725 - val_loss: 0.3432 - val_accuracy: 0.8888 - lr: 0.0010
Epoch 7/8
444/444 [==============================] -

## Phase 2, fine-tuning

1) Load the best phase-1 weights

In [36]:
import tensorflow as tf

model = tf.keras.models.load_model("../models/v3_best.keras", safe_mode=False)
print("Loaded:", "../models/v3_best.keras")

Loaded: ../models/v3_best.keras


2) Unfreeze the backbone partially

In [37]:
# Find the base model by name (we created it as EfficientNetB0)
base_model = None
for layer in model.layers:
    if isinstance(layer, tf.keras.Model) and "efficientnet" in layer.name.lower():
        base_model = layer
        break

print("Base model:", base_model.name)

base_model.trainable = True

# Unfreeze only top N layers (start with 20)
N = 80
for layer in base_model.layers[:-N]:
    layer.trainable = False

print("Trainable layers in base_model:", sum(l.trainable for l in base_model.layers), "/", len(base_model.layers))


Base model: efficientnetb0
Trainable layers in base_model: 80 / 239


3) Recompile with a very small learning rate

In [38]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)


4) Training, fine-tune for a few epochs

In [39]:
history2 = model.fit(
    mixed_train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks
)


Epoch 1/10
444/444 [==============================] - 373s 818ms/step - loss: 0.3143 - accuracy: 0.8983 - val_loss: 0.2519 - val_accuracy: 0.9190 - lr: 1.0000e-05
Epoch 2/10
444/444 [==============================] - 368s 828ms/step - loss: 0.2565 - accuracy: 0.9160 - val_loss: 0.2139 - val_accuracy: 0.9323 - lr: 1.0000e-05
Epoch 3/10
444/444 [==============================] - 372s 836ms/step - loss: 0.2152 - accuracy: 0.9308 - val_loss: 0.1908 - val_accuracy: 0.9376 - lr: 1.0000e-05
Epoch 4/10
444/444 [==============================] - 369s 829ms/step - loss: 0.1848 - accuracy: 0.9383 - val_loss: 0.1705 - val_accuracy: 0.9464 - lr: 1.0000e-05
Epoch 5/10
444/444 [==============================] - 382s 858ms/step - loss: 0.1669 - accuracy: 0.9455 - val_loss: 0.1568 - val_accuracy: 0.9498 - lr: 1.0000e-05
Epoch 6/10
444/444 [==============================] - 373s 840ms/step - loss: 0.1461 - accuracy: 0.9527 - val_loss: 0.1584 - val_accuracy: 0.9486 - lr: 1.0000e-05
Epoch 7/10
444/444 [==

To control whether all image files are ok:

In [22]:
from pathlib import Path

def find_zero_byte(paths, name):
    bad = []
    for p in paths:
        p = Path(p)
        try:
            if p.exists() and p.stat().st_size == 0:
                bad.append(str(p))
        except OSError:
            bad.append(str(p))
    print(f"{name}: {len(bad)} zero-byte/unreadable files")
    for b in bad[:20]:
        print("  ", b)
    return bad

bad_pv = find_zero_byte(train_paths, "PV train")
bad_wt = find_zero_byte(wild_train_paths, "Wild train")
bad_wv = find_zero_byte(wild_val_paths, "Wild val")


PV train: 0 zero-byte/unreadable files
Wild train: 0 zero-byte/unreadable files
Wild val: 0 zero-byte/unreadable files


In [25]:
import tensorflow as tf
from pathlib import Path

def find_first_bad(paths, name="set"):
    for p in paths:
        p = Path(p)
        try:
            raw = tf.io.read_file(str(p))
            # Force eager conversion so errors surface here
            raw_bytes = raw.numpy()
            if len(raw_bytes) == 0:
                print(f"[{name}] EMPTY READ:", p)
                return str(p)
            img = tf.image.decode_image(raw, channels=3, expand_animations=False)
            _ = img.numpy()  # force decode
        except Exception as e:
            print(f"[{name}] BAD FILE:", p)
            print("Reason:", repr(e))
            return str(p)
    print(f"[{name}] No bad files found.")
    return None

# bad = find_first_bad(wild_train_paths, "wild_train")
# If none found, try mixed sources:
# bad = find_first_bad(train_paths, "pv_train")
bad = find_first_bad(wild_val_paths, "wild_val")


[wild_val] No bad files found.
